# Text Generation with RNN

* RNN을 사용하여 어떻게 텍스트를 생성하는지 알아보자.
* 주어진 텍스트를 기반으로 텍스트를 생성하는 모델을 구현해보자.

### 생성의 한계

문장 중 일부는 문법적으로 맞지만 대부분 자연스럽지 않다.

이 모델은 단어의 의미를 학습하지는 않았지만, 고려해야 할 점으로:

* 데이터는 문자 기반이다. 훈련이 시작되었을 때, 이 모델은 영어 단어의 철자를 모르고 심지어 텍스트의 단위가 단어라는 것도 모른다.

## 설정

### Drive 연결


In [ ]:
use_colab = True
assert use_colab in [True, False]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf

import numpy as np
import os
import time

### 셰익스피어 데이터셋 다운로드

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

### 데이터 읽기

In [ ]:
# 데이터를 불러와서 디코딩
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# 문자의 수
print ('텍스트의 길이: {}'.format(len(text)))

In [ ]:
# 텍스트 처음 250자 출력
print(text[1500:2500])

In [ ]:
# 파일의 고유 문자수를 출력
vocab = sorted(set(text)) # 내가 불러온 text 데이터를 집합으로 만들어서 정렬시킨 상태입니다.
print ('고유 문자수 {}개'.format(len(vocab)))

## 텍스트 처리

### 텍스트 벡터화

* 학습을 위해서 텍스트들을 수치화할 필요가 있다.
* 텍스트를 인덱스화 시켜 학습에 사용

In [ ]:
# 고유 문자에서 인덱스로 매핑 생성
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab) # idx <=> char 변환할 수 있는 사전을 들고있는 것이 중요합니다.

text_as_int = np.array([char2idx[c] for c in text]) # 람다식

In [ ]:
idx2char

In [ ]:
print(text_as_int)
text_as_int.shape

* 텍스트 0번부터 전체 텍스트 길이까지 인덱스화

In [ ]:
print('{')
for char,_ in zip(char2idx, range(65)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))


In [ ]:
# 텍스트 맵핑
print ('{} ---- Index ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

### 예측 과정

주어진 문자나 문자 시퀀스가 주어졌을 때, 다음 문자로 가장 가능성 있는 문자는 무엇일까?

* 이는 모델을 훈련하여 수행할 작업이다.
* 모델의 입력은 문자열 시퀀스가 될 것이고, 모델을 훈련시켜 출력을 예측한다.
* 이 출력은 현재 타임 스텝(time step)의 다음 문자이다.

### 훈련 샘플과 타깃 만들기

* 다음으로 텍스트를 샘플 시퀀스로 나누자.

* 각 입력 시퀀스에는 텍스트에서 나온 `seq_length`개의 문자가 포함된다.

* 각 입력 시퀀스에서, 해당 타깃은 한 문자를 오른쪽으로 이동한 것을 제외하고는 동일한 길이의 텍스트를 포함한다.

* 텍스트를`seq_length + 1`개의 청크(chunk)로 나누자
    * 예를 들어, `seq_length`는 4이고 텍스트를 "Hello"이라고 가정해 봅시다. 입력 시퀀스는 "Hell"이고 타깃 시퀀스는 "ello"가 된다.

* 이렇게 하기 위해 먼저 `tf.data.Dataset.from_tensor_slices` 함수를 사용해 텍스트 벡터를 문자 인덱스의 스트림으로 변환한다.

In [ ]:
# Hello 라는 단어를 만들고 싶습니다.
# 학습 데이터를 아래와 같이 세팅을 해줘야합니다.
# input : H -> e -> l -> l
# output : e -> l -> l -> o

In [ ]:
# 단일 입력에 대해 원하는 문장의 최대 길이
seq_length = 100
examples_per_epoch = len(text)//seq_length

# 훈련 샘플/타깃 만들기
char_dataset = tf.data.Dataset.from_tensor_slices(#) # 알파벳을 하나씩 생성합니다.

for i in char_dataset.take(5):
    print(i.numpy())
    print(idx2char[i.numpy()])#확인가능

* `batch`를 이용해 몇개의 텍스트를 가져올 것인지 정할 수 있다.

In [ ]:
sequences = char_dataset.batch(#, drop_remainder=True) # 문장을 가져와합니다.
                               # 문장을 가져올 수 있도록 batch 를 구성해줍니다.

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()]))) # repr 개행문자 출력

각 시퀀스에서, `map` 메서드를 사용해 각 배치에 간단한 함수를 적용하고 입력 텍스트와 타깃 텍스트를 복사 및 이동
* 모델이 Text를 생성할때, 한 텍스트 단위로 생성하기 때문에 그 다음 텍스트를 target으로 사용

In [ ]:
# Hello -> 불러온 데이터의 길이
# input : Hell # input의 길이
# output : ello # output의 길이

def split_input_target(chunk):
    input_text = #
    target_text = #
    return input_text, target_text

dataset = sequences.map(split_input_target)

첫 번째 샘플의 타깃 값을 출력해보자

In [ ]:
for input_example, target_example in  dataset.take(1):
    print ('입력 데이터: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('타깃 데이터: ', repr(''.join(idx2char[target_example.numpy()])))

* 이 벡터의 각 인덱스는 하나의 타임 스텝(time step)으로 처리됩니다. 타임 스텝 0의 입력으로 모델은 "F"의 인덱스를 받고 다음 문자로 "i"의 인덱스를 예측한다.

* 다음 타임 스텝에서도 같은 일을 하지만 RNN은 현재 입력 문자 외에 이전 타임 스텝의 컨텍스트**(context)**를 고려한다.

In [ ]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("iter {:4d}".format(i))
    print("  inputs: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  generated text: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

### 훈련 배치 생성

* 텍스트를 다루기 쉬운 시퀀스로 분리하기 위해 `tf.data`를 사용
* 이 데이터를 모델에 넣기 전에 데이터를 섞은 후 배치를 만들어야 한다.

In [ ]:
# 배치 크기
BATCH_SIZE = #

dataset = dataset.shuffle(10000).batch(BATCH_SIZE, drop_remainder=True)

dataset
# 알파벳을 기준으로 했던 데이터에서 문장단위로 데이터를 불러오는 dataset을 구성할 수 있습니다.

## 모델 설계

모델을 정의하려면 `tf.keras.Sequential`을 사용한다.

이 예제에서는 3개의 층을 사용하여 모델을 정의한다:

* `tf.keras.layers.Embedding` : 입력층. `embedding_dim` 차원 벡터에 각 문자의 정수 코드를 매핑하는 훈련 가능한 검색 테이블.
* `tf.keras.layers.GRU` : 크기가 `units = rnn_units`인 RNN의 유형(여기서 LSTM층을 사용할 수도 있다.)
* `tf.keras.layers.Dense` : 크기가 `vocab_size`인 출력을 생성하는 출력층.

각 문자에 대해 모델은 임베딩을 검색하고, 임베딩을 입력으로 하여 GRU를 1개의 타임 스텝으로 실행하고, FC layers를 적용하여 다음 문자의 로그 가능도(log-likelihood)를 예측하는 로짓을 생성한다:

In [ ]:
# 문자로 된 어휘 사전의 크기
vocab_size = #TODO

# 임베딩 차원
embedding_dim = # TODO

# RNN 유닛(unit) 개수
rnn_units = # TODO

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(#TODO, #TODO,
                                batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(#TODO,
                            return_sequences=#,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(#TODO)
    ])

    return model

In [ ]:
model = build_model(
    vocab_size = #,
    embedding_dim=#,
    rnn_units=#,
    batch_size=BATCH_SIZE)

In [ ]:
for layer in model.layers:
    print(layer.output_shape)

## 모델 사용

이제 모델을 실행하여 원하는대로 동작하는지 확인해보자.

먼저 출력의 형태를 확인하자.

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (배치 크기, 시퀀스 길이, 어휘 사전 크기)")

위 예제에서 입력의 시퀀스 길이는 100이지만 모델은 임의 시퀀스 길이의 입력도 사용 가능하다.

In [ ]:
model.summary()

## 모델 훈련

이 문제는 표준 분류 문제로 취급될 수 있습니다. 이전 RNN 상태와 이번 타임 스텝(time step)의 입력으로 다음 문자의 클래스를 예측한다.

### Optimizer, Loss function

`tf.keras.losses.sparse_softmax_crossentropy` 를 사용해 label을 벡터로 바꾸지 않고 loss를 계산한다.

이 모델은 로짓을 반환하기 때문에 `from_logits` 플래그를 설정해야 한다.

In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=#)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("예측 배열 크기(shape): ", example_batch_predictions.shape, " # (배치 크기, 시퀀스 길이, 어휘 사전 크기)")
print("Loss: ", example_batch_loss.numpy().mean())

### 학습준비
* `tf.keras.Model.compile` 메서드를 사용하여 훈련 절차를 설정
* 기본 매개변수의 `tf.keras.optimizers.Adam`과 손실 함수를 사용

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(#TODO),
              loss=loss)

### 체크포인트 구성

`tf.keras.callbacks.ModelCheckpoint`를 사용하여 훈련 중 체크포인트(checkpoint)가 저장되도록 설정한다.

In [ ]:
# the save point
if use_colab:
    checkpoint_dir ='./drive/My Drive/train_ckpt/text_gen/exp1'
    if not os.path.isdir(checkpoint_dir):
        os.makedirs(checkpoint_dir)
else:
    checkpoint_dir = 'text_gen/exp1'

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_dir,
                                                 save_weights_only=True,
                                                 monitor='loss',
                                                 mode='auto',
                                                 save_best_only=True,
                                                 verbose=1)

### 훈련 실행

In [ ]:
EPOCHS=#TODO 5~10 에폭정도

In [ ]:
history = model.fit(#TODO,
                    epochs=#TODO,
                    callbacks=[cp_callback])

## 텍스트 생성

### 최근 체크포인트 복원

이 예측 단계에선 Batch size 1을 사용한다.

* RNN 상태가 타임 스텝에서 타임 스텝으로 전달되는 방식이기 때문에 모델은 한 번 빌드된 고정 배치 크기만 허용
* 다른 배치 크기로 모델을 실행하려면 모델을 다시 빌드하고 체크포인트에서 가중치를 복원해야 한다.

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(checkpoint_dir)

model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

### 예측 루프

텍스트 생성:

* 시작 문자열 선택과 순환 신경망 상태를 초기화하고 생성할 문자 수를 설정

* 시작 문자열과 순환 신경망 상태를 사용하여 다음 문자의 예측 배열을 가져온다.

* 다음, 범주형 배열을 사용하여 예측된 문자의 인덱스를 계산

* 이 예측된 문자를 모델의 다음 입력으로 활용

* 모델에 의해 리턴된 RNN 상태는 모델로 피드백되어 이제는 하나의 단어가 아닌 더 많은 컨텍스트를 갖추게 된다.

* 다음 단어를 예측한 후 수정된 RNN 상태가 다시 모델로 피드백되어 이전에 예측된 단어에서 더 많은 컨텍스트를 얻으면서 학습하는 방식

![텍스트를 생성하기 위해 모델의 출력이 입력으로 피드백](https://tensorflow.org/tutorials/text/images/text_generation_sampling.png)

* 생성된 텍스트를 보면 모델이 언제 대문자로 나타나고, 절을 만들고 셰익스피어와 유사한 어휘를 가져오는지 볼 수 있다.

In [ ]:
def generate_text(model, start_string):
  # 평가 단계 (학습된 모델을 사용하여 텍스트 생성)

  # 생성할 문자의 수
  num_generate = 1000

  # 시작 문자열을 숫자로 변환(벡터화)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # 결과를 저장할 빈 문자열
  text_generated = []

  # 온도가 낮으면 더 예측 가능한 텍스트 생성
  # 온도가 높으면 더 의외의 텍스트 생성 (불확실성)
  # 최적의 세팅을 찾기 위한 실험
  temperature = 1.0

  # 여기에서 배치 크기 == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # 배치 차원 제거
      predictions = tf.squeeze(predictions, 0)

      # 범주형 분포를 사용하여 모델에서 리턴한 단어 예측
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # 예측된 단어를 다음 입력으로 모델에 전달
      # 이전 은닉 상태와 함께
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"ROMEO: "))